In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import multiprocessing as mp
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.optim import Adam
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
os.environ["CUDA_VISIBLE_DEVICES"] = '3'

In [2]:
trained_model_path = '/workdir/security/home/junjiehuang2468/paper/trained_models_weight/ember/'
best_trained_model = '2022-01-18 14:55/2w_epoch:0_test_acc:0.890858.pt'
# best_trained_model = '2022-01-24 17:46/2w_epoch:5_test_acc:0.895942.pt'
data_path = "/workdir/security/home/junjiehuang2468/paper/data/ember2018/"
train_data_path = data_path + "malwares/" 
test_data_path = data_path + "test_malwares/" 

In [3]:
CUDA = True if torch.cuda.is_available() else False
NUM_WORKERS = 24
BATCH_SIZE = 40
# BATCH_SIZE = 10
LEAVE_BIT_NUMBER = 20000
KERNEL_SIZE = 500

In [4]:
LEAVE_BIT_NUMBER_DICT = {10000:'1w',20000:'2w'}

In [5]:
trainset = pd.read_csv(data_path + 'train_dataset.csv')
validset = pd.read_csv(data_path + 'valid_dataset.csv')
testset = pd.read_csv(data_path + 'test_dataset.csv')
testset = testset.iloc[np.argwhere(testset['labels'].values == 1).squeeze(),:]

In [6]:
class ExeDataset(Dataset):
    def __init__(self, malware_names, data_path, labels, leave_bit_num):
        self.malware_names = malware_names
        self.data_path = data_path
        self.labels = labels
        self.leave_bit_num = leave_bit_num

    def __len__(self):
        return len(self.malware_names)

    def __getitem__(self, idx):
        with open(self.data_path + self.malware_names[idx] + '.txt','rb') as fp:
            data = [bit+1 for bit in fp.read()[:self.leave_bit_num]]
            padding = [0]*(self.leave_bit_num-len(data))
            data = data + padding

        return np.array(data), np.array([self.labels[idx]])

In [7]:
train_dataset = ExeDataset(
    trainset["id"].tolist(), 
    train_data_path, 
    trainset["labels"].tolist(), 
    LEAVE_BIT_NUMBER
)
valid_dataset = ExeDataset(
    validset["id"].tolist(), 
    train_data_path, 
    validset["labels"].tolist(), 
    LEAVE_BIT_NUMBER
)
test_dataset = ExeDataset(
    testset["id"].tolist(), 
    test_data_path, 
    testset["labels"].tolist(), 
    LEAVE_BIT_NUMBER
)

In [8]:
trainloader = DataLoader(
    dataset = train_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False,
    num_workers = NUM_WORKERS,
    pin_memory = True
)
validloader = DataLoader(
    dataset = valid_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False,
    num_workers = NUM_WORKERS,
    pin_memory = True
)
testloader = DataLoader(
    dataset = test_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False,
    num_workers = NUM_WORKERS,
    pin_memory = True
)

In [9]:
class Model(nn.Module):
    def __init__(self, data_length = 2e6, kernel_size = 500):
        super().__init__()
        self.embedding = nn.Embedding(257, 8, padding_idx=0)
        self.conv_layer_1 = nn.Conv1d(4, 128, kernel_size, stride = kernel_size, bias = True)
        # self.bn_1 = nn.BatchNorm1d(128)
        self.conv_layer_2 = nn.Conv1d(4, 128, kernel_size, stride = kernel_size, bias = True)
        self.pool_layer_2 = nn.MaxPool1d(data_length//kernel_size)
        self.fc_layer_3 = nn.Linear(128, 128)
        self.fc_layer_4 = nn.Linear(128, 2)
        
    def sequence(self,embedd_x):
        x = embedd_x.transpose(-1,-2)
        x_conv_1 = self.conv_layer_1(x[:,:4,:])
        x_conv_2 = torch.sigmoid(self.conv_layer_2(x[:,4:,:]))
        x = x_conv_1*x_conv_2
        del x_conv_1,x_conv_2
        x = self.pool_layer_2(x).squeeze()
        x = self.fc_layer_3(x)
        x = self.fc_layer_4(x)
        return x
        
    def forward(self, input_, loss_fn):
        batch_acc = []
        batch_grad = []
        
        embedd_x = self.embedding(input_)
        x = self.sequence(embedd_x)
        acc = torch.argmax(torch.softmax(x,dim=-1),dim=-1).float().mean()
        batch_acc.append(acc.tolist())
        
        padding_place_mask = input_ == 0
        random_padding = torch.randint_like(input=padding_place_mask.float(),low=1,high=257) * padding_place_mask.float()
        input_ += random_padding.long()
        
        one_hot_x = F.one_hot(input_,num_classes=257).float()
        one_hot_x.requires_grad = True
        one_hot_x.retain_grad()
        for _ in range(13):
            embedd_x = one_hot_x @ self.embedding.weight
            x = self.sequence(embedd_x)
            
            acc = torch.argmax(torch.softmax(x,dim=-1),dim=-1).float().mean()
            batch_acc.append(acc.tolist())
            
            mislead_labels = torch.cat((torch.ones(len(x)).reshape(-1,1),torch.zeros(len(x)).reshape(-1,1)),dim=1).cuda()
            loss = loss_fn(x,mislead_labels)
            loss.backward()
            
            shape = padding_place_mask.shape
            expand_padding_place_mask = padding_place_mask.unsqueeze(-1).expand(shape[0],shape[1],257)
            grad_min_idx = torch.argmin(one_hot_x.grad,dim=-1)
            grad_sez_mask = (one_hot_x.grad <= 0)
            grad_mask = (expand_padding_place_mask & grad_sez_mask)
            batch_grad.append(torch.div(
                input=(expand_padding_place_mask.float()*one_hot_x.grad).sum(),
                other=expand_padding_place_mask.float().sum()
            ).detach().cpu().numpy())
            
            one_hot_x.data = F.one_hot(torch.argmin(
                one_hot_x.grad,
                dim=-1
            ),num_classes=257).float()*(expand_padding_place_mask.float()) + one_hot_x*(1 - expand_padding_place_mask.float())
            
            one_hot_x.grad.zero_()
            
        embedd_x = one_hot_x @ self.embedding.weight
        x = self.sequence(embedd_x)
        return x.detach().cpu().numpy(),batch_acc,batch_grad,one_hot_x

In [10]:
model = Model(data_length=LEAVE_BIT_NUMBER,kernel_size=KERNEL_SIZE)

ce_loss = nn.CrossEntropyLoss()
optim = Adam(model.parameters())

model = model.cuda() if CUDA else model
ce_loss = ce_loss.cuda() if CUDA else ce_less

In [11]:
model.load_state_dict(torch.load(trained_model_path + best_trained_model))

<All keys matched successfully>

In [ ]:
model.eval()
acc = []
preds = []
labels = []
total_batch_acc = []
total_batch_grad = []
bar = tqdm(testloader)
# bar = tqdm(validloader)
for step, (batch_data,batch_label) in enumerate(bar):
    batch_data = batch_data.cuda() if CUDA else batch_data
    batch_label = batch_label.cuda() if CUDA else batch_label
    batch_label = batch_label.squeeze()
    temp_label = torch.zeros((len(batch_label),2))
    for idx,target in enumerate(batch_label.squeeze()): temp_label[idx,target] = 1
    temp_label = temp_label.cuda() if CUDA else temp_label

#     pred,input_,one_hot_x = model(batch_data,ce_loss)
#     break
    pred,batch_acc,batch_grad,one_hot_x = model(batch_data,ce_loss)
#     break
    pred = np.argmax(pred,1)
    batch_label = batch_label.cpu().data.numpy()
    
    preds.extend(pred.tolist())
    labels.extend(batch_label.tolist())
    
    temp_acc = (batch_label == pred).mean()
    acc.append(temp_acc)
    total_batch_acc.append(batch_acc + [temp_acc])
    total_batch_grad.append([0] + batch_grad)
    total_batch_acc_str = '[' + ' '.join(map(lambda x: '%.10f'%x,np.mean(total_batch_acc,axis=0))) + ']'
    total_batch_grad_str = '[' + ' '.join(map(lambda x: '%.10f'%x,np.mean(total_batch_grad,axis=0))) + ']'
    print(total_batch_acc_str)
    print(total_batch_grad_str)
    print(f"test：{temp_acc:4f}, test mean: {np.mean(acc):4f}")

  0%|          | 1/2500 [00:03<2:07:47,  3.07s/it]

[0.7750000358 0.3499999940 0.0000000000 0.7750000358 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0500000007 0.0000000000 0.0000000000 0.0000000000]
[0.0000000000 -0.0000001789 0.0000000000 -0.0000000548 0.0000000000 -0.0000000000 0.0000000000 -0.0000000000 0.0000000000 -0.0000000000 0.0000000000 -0.0000000017 0.0000000000 -0.0000000000]
test：0.000000, test mean: 0.000000


  0%|          | 2/2500 [00:03<1:14:13,  1.78s/it]

[0.7875000238 0.4000000060 0.0000000000 0.7875000238 0.0250000004 0.0250000004 0.0250000004 0.0250000004 0.0250000004 0.0250000004 0.0250000004 0.0625000019 0.0250000004 0.0250000004 0.0250000000]
[0.0000000000 -0.0000002011 0.0000000000 -0.0000000540 0.0000000002 0.0000000002 0.0000000003 0.0000000002 -0.0000000000 0.0000000000 -0.0000000000 -0.0000000010 0.0000000000 0.0000000002]
test：0.050000, test mean: 0.025000


  0%|          | 3/2500 [00:04<56:01,  1.35s/it]  

...

[0.8413134536 0.4047127790 0.0041968768 0.8389291814 0.0213887344 0.0140267710 0.0131204687 0.0145147799 0.0127997771 0.0165644175 0.0126185167 0.0173034024 0.0126185167 0.0164668157 0.0125627440]
[0.0000000000 -0.0000001696 0.0000000000 -0.0000000585 0.0000000005 0.0000000001 -0.0000000000 -0.0000000000 -0.0000000000 -0.0000000005 -0.0000000000 -0.0000000005 -0.0000000000 -0.0000000005]
test：0.050000, test mean: 0.012563


 72%|███████▏  | 1794/2500 [25:01<09:53,  1.19it/s]

[0.8412764896 0.4045847340 0.0041945374 0.8388935464 0.0213907473 0.0140328876 0.0131131552 0.0145066892 0.0127926424 0.0165551842 0.0126114829 0.0173076926 0.0126114829 0.0164576369 0.0125557414]
[0.0000000000 -0.0000001696 0.0000000000 -0.0000000585 0.0000000005 0.0000000001 -0.0000000000 -0.0000000000 -0.0000000000 -0.0000000005 -0.0000000000 -0.0000000005 -0.0000000000 -0.0000000005]
test：0.000000, test mean: 0.012556


 72%|███████▏  | 1795/2500 [25:02<09:51,  1.19it/s]

[0.8412813495 0.4045264695 0.0041922006 0.8388997339 0.0213927581 0.0140250699 0.0131058498 0.0144986075 0.0127855155 0.0165459613 0.0126044570 0.0172980504 0.0126044570 0.0164623958 0.0125487465]
[0.0000000000 -0.0000001696 0.0000000000 -0.0000000585 0.0000000005 0.0000000001 -0.0000000000 -0.0000000000 -0.0000000000 -0.0000000005 -0.0000000000 -0.0000000005 -0.0000000000 -0.0000000005]
test：0.000000, test mean: 0.012549


In [ ]:
# import pickle

In [ ]:
# with open('/workdir/security/home/junjiehuang2468/paper/results/ember/my_way/cumulative_gradients_v4_13iter_acc.txt','wb') as fp:
#     pickle.dump(total_batch_acc,fp)